In [ ]:
import lyricsgenius
import pandas as pd
import time

genius_token = "Q7F1UAjgD74fWuudQiukK18h-wK6fO-KRAOiPE_8b2WTfCn_HGM5ucMZE_Ziuzbe"

In [ ]:
genius = lyricsgenius.Genius(genius_token)

In [ ]:
# Function to fetch lyrics
def fetch_lyrics(song_title, artist_name):
    try:
        # Search for the song
        song = genius.search_song(song_title, artist_name)
        if song:
            print(f"Fetched lyrics for '{song_title}' by '{artist_name}'")
            return song.lyrics
        else:
            print(f"No lyrics found for '{song_title}' by '{artist_name}'")
            return None
    except Exception as e:
        print(f"Error fetching lyrics for '{song_title}' by '{artist_name}': {e}")
        return None


In [ ]:
# Read the data
song_list = pd.read_csv('songlist.csv')

# Clean the data
song_list = song_list.drop(columns=["Unnamed: 0"])
song_list["Year"] = song_list["Year"].astype(int) 

# Find where it gets bad 
error_index = song_list[song_list['Year'] == 2021].index[0]

# Split the data
df_correct = song_list.iloc[:error_index]
df_mistake = song_list.iloc[error_index:]

# Split 'Artist' column into 'Artist' and 'Additional Text'
df_mistake[['Song2', 'Artist']] = df_mistake['Artist'].str.split('and', n=1, expand=True)
df_mistake['Song2'] = df_mistake['Song2'].str.replace('"', '')
df_mistake["Artist"] = df_mistake['Artist'].str[:-3]
df_correct["Artist"] = df_correct['Artist'].str[:-3]
df_mistake["Song"] = df_mistake["Song2"]
df_mistake = df_mistake.drop(columns=["Song2"])

song_list = pd.concat([df_correct, df_mistake], ignore_index=True)

In [ ]:
song_list

In [ ]:
# Dictionary to store lyrics
lyrics_dict = {}

# Iterate over songs
for index, row in song_list.iterrows():
    song_title = row['Song']
    artist_name = row['Artist']
    
    # Fetch lyrics
    lyrics = fetch_lyrics(song_title, artist_name)
    if lyrics:
        lyrics_dict[f"{song_title} - {artist_name}"] = lyrics
    
    # Sleep to handle rate limiting
    time.sleep(1)

In [ ]:
import csv

with open('lyrics.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Song Title', 'Lyrics'])
    for song_info, lyrics in lyrics_dict.items():
        writer.writerow([song_info, lyrics])